In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
autolib = pd.read_csv("autolib_daily.csv")
autolib.head (10)

,Postal code,date,n_daily_data_points,dayOfWeek,day_type,BlueCars_taken_sum,BlueCars_returned_sum,Utilib_taken_sum,Utilib_returned_sum,Utilib_14_taken_sum,Utilib_14_returned_sum,Slots_freed_sum,Slots_taken_sum
0,75001,1/1/2018,1440,0,weekday,110,103,3,2,10,9,22,20
1,75001,1/2/2018,1438,1,weekday,98,94,1,1,8,8,23,22
2,75001,1/3/2018,1439,2,weekday,138,139,0,0,2,2,27,27
3,75001,1/4/2018,1320,3,weekday,104,104,2,2,9,8,25,21
4,75001,1/5/2018,1440,4,weekday,114,117,3,3,6,6,18,20
5,75001,1/6/2018,1437,5,weekend,187,185,6,6,7,8,38,35
6,75001,1/7/2018,1440,6,weekend,180,180,2,2,10,9,34,34
7,75001,1/8/2018,1438,0,weekday,84,83,3,3,10,10,14,15
8,75001,1/9/2018,1439,1,weekday,81,84,1,1,4,4,15,15
9,75001,1/10/2018,1440,2,weekday,88,85,5,5,11,11,23,22


In [9]:
#Exploratory Data Analysis

import pandas_profiling
pandas_profiling.ProfileReport(autolib)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,13
Number of observations,16085
Total Missing (%),0.0%
Total size in memory,1.6 MiB
Average record size in memory,104.0 B
Numeric,5
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,6


In [10]:
#Selecting random sample to get a different view of the data

random_subset = autolib.sample(n=10)
random_subset.head()

,Postal code,date,n_daily_data_points,dayOfWeek,day_type,BlueCars_taken_sum,BlueCars_returned_sum,Utilib_taken_sum,Utilib_returned_sum,Utilib_14_taken_sum,Utilib_14_returned_sum,Slots_freed_sum,Slots_taken_sum
12697,93700,2/12/2018,1440,0,weekday,26,25,4,3,3,3,0,0
9955,92800,6/8/2018,1438,4,weekday,104,108,3,3,2,1,10,10
11315,93300,5/31/2018,1411,3,weekday,7,6,0,0,1,1,0,0
9363,92500,1/21/2018,1440,6,weekend,167,161,6,5,13,14,0,0
3715,78140,5/23/2018,1174,2,weekday,31,30,2,2,0,0,0,0


# Our NULL and ALTERNATE hypotheses

Bourg-la-Reine and Bagnolet are two communes located in PAris, France.

H0: The Bourg-la-Reine commune in Paris, benefits from being a residential area and as a result sees more (or equal) electric car rental activity than Bagnolet.

H1: Bagnolet, by virtue of being a more metropolitan area of Paris, sees more electric car rental activity compared to Bourg-la-Reine which is a more suburban area in nature.

--put in other terms--

H0: Bourg-la-Reine >= Bagnolet

H1: Bourg-la-Reine < Bagnolet


# Cleaning

In [11]:
#Dropping null values, since they will interfere with calculations in our key column (Blue Cars Taken)

autolib['BlueCars_taken_sum'].dropna()

0        110
1         98
2        138
3        104
4        114
        ... 
16080     15
16081     19
16082     33
16083     11
16084      2
Name: BlueCars_taken_sum, Length: 16085, dtype: int64

In [0]:
#Fixing column names containing spaces
autolib.columns = autolib.columns.str.strip().str.lower().str.replace(' ', '_')

In [13]:
autolib.head(5)

,postal_code,date,n_daily_data_points,dayofweek,day_type,bluecars_taken_sum,bluecars_returned_sum,utilib_taken_sum,utilib_returned_sum,utilib_14_taken_sum,utilib_14_returned_sum,slots_freed_sum,slots_taken_sum
0,75001,1/1/2018,1440,0,weekday,110,103,3,2,10,9,22,20
1,75001,1/2/2018,1438,1,weekday,98,94,1,1,8,8,23,22
2,75001,1/3/2018,1439,2,weekday,138,139,0,0,2,2,27,27
3,75001,1/4/2018,1320,3,weekday,104,104,2,2,9,8,25,21
4,75001,1/5/2018,1440,4,weekday,114,117,3,3,6,6,18,20


# Hypothesis Testing

In [0]:
#CHeck number of occurences in our dataset for the two communes that we are testing.
#Knowing these numbers will help us determine our sample sizes ("n")


In [31]:
# Create dataframe for the two data sets we want to compare
df_92340 =  autolib[autolib['postal_code'] == 92340 ]
df_93170=  autolib[autolib['postal_code'] == 93170 ]

print("Bourg-la-Reine i.e df_92340 has ",df_92340.shape)
print("Bagnolet i.e df_93170 has ",df_93170.shape)
print(autolib.shape)


Bourg-la-Reine i.e df_92340 has  (147, 13)
Bagnolet i.e df_93170 has  (156, 13)
(16085, 13)


In [64]:
re = df_92340['bluecars_taken_sum'].sum()
ww = df_93170['bluecars_taken_sum'].sum()

shape_re = df_92340['bluecars_taken_sum'].shape[0]
shape_ww = df_92340['bluecars_taken_sum'].shape[0]

print (re)
print (ww)

1845
6917


Occurences for both range between 147 and 156.  We can use a sample size that's close to 50%. We will use **n = 75**. That means we will pick 75 random samples from our data to use for hypothesis testing.


We will use a **z_score** as the test statistic since our sample size contains more than 30 elements.

My chosen **level of significance** is **0.05**

In [0]:
# Sample
# Sample 1
df_92340_sample = df_92340['bluecars_taken_sum'].sample(75)

# Sample 2
df_93170_sample = df_93170['bluecars_taken_sum'].sample(75)


In the next cell I tried to do the z score calculation manually. As I learned, Things can get pretty thick if you try doing this with two samples. 

In [51]:


# z score calculation
# Formula is x1-x2/sqrt(std1/n1+std2/n2)
import math

x1 = df_92340_sample.mean()
x2 = df_93170_sample.mean()
print( "x1 mean is " ,x1 ," x2 mean is ", x2)

std1 = df_92340_sample.std()
std2 = df_93170_sample.std()
print( "std1  is " ,std1 ," std2  is ", std2)

n1 = df_92340_sample.shape[0]
n2 = df_93170_sample.shape[0]

z_score = (x1 - x2)/math.sqrt((std1/n1)+(std2/n2))

z_score


x1 mean is  12.146666666666667  x2 mean is  43.4
std1  is  6.321762783012718  std2  is  13.152946437965904


-61.332615406864065

After hitting a dead end with the manual calculation, I was able to find a custom function that suits my needs here.

The results are detailed in the next cell.

In [0]:
def calculate_zscore(sample1, sample2):
    from statsmodels.stats import weightstats as stests #For z test calculation
    ztest ,pval1 = stests.ztest(sample1, x2=sample2, value=0,alternative='two-sided')
    return (ztest ,pval1)

In [62]:
calculate_zscore(df_92340_sample, df_93170_sample)

(-18.546978983316237, 8.626621621551957e-77)

# Testing results
The z score is -18.546

The p value is 8.626621621551957e-77

Compared to our level of significance stated earlier, the p value is much higher.


This means that our test is statistically insignificant. It also means that we reject the alternate hypothesis.

Random sampling is not a 'silver bullet' for hypothesis testing. The samples may be chosen in a way that makes the sample set skew one way, i.e. bias is introduced.

Stratification of the population can help mitigate this, but even that is not foolproof. 

GOOD PRACTICE IS to  QUESTION the results of the hypothesis testing and iterate with improvements, as opposed to completely abandoning the entire premise. A rejection of the alternate hypothesis is not equal to complete failure.